# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [50]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, concat_ws, collect_list, lag
se = SparkSession(sc)

2024-01-24 19:49:28,044 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


### Spark DF solution

In [51]:
df = se.read.csv("hdfs:/data/clickstream.csv", header=True, sep="\t")

In [52]:
logs = df.withColumn('timestamp', col('timestamp').cast('int'))

In [53]:
errors = logs.filter(logs['event_type'].contains('error')).groupby(['user_id', 'session_id']).min('timestamp')

In [54]:
logs = logs.join(errors, ['user_id', 'session_id'], 'left').withColumnRenamed('min(timestamp)', 'error_time')

In [55]:
logs = logs.filter(((logs['timestamp'] < logs['error_time']) | (logs['error_time'].isNull())) & (logs['event_type'] == 'page'))

In [56]:
window  = Window.partitionBy(['user_id','session_id']).orderBy('timestamp')

In [57]:
logs = logs.withColumn('previous_page', lag('event_page', 1).over(window))

In [58]:
logs = logs.filter((logs['event_page'] != logs['previous_page']) | (logs['previous_page'].isNull()))

In [59]:
routes = logs.groupby(['user_id','session_id']).agg(concat_ws('-', collect_list('event_page')).alias('route'))

In [60]:
result_df = routes.groupby(['route']).count()

In [61]:
result_df = result_df.sort('count', ascending=False).take(30)

In [62]:
with open('result_df.txt', 'w') as f:
    for line in result_df:
         f.write("{}\t{}\n".format(line[0], line[1]))

### Spark SQL solution

In [79]:
df.registerTempTable('table_logs')

In [80]:
routes_sql = se.sql('select t.user_id, t.session_id, t.event_page, t.timestamp, lag(t.event_page, 1) over (partition by t.user_id, t.session_id order by t.timestamp) as previous_page \
from table_logs as t left join (select user_id, session_id, min(timestamp) as error_time from table_logs where event_type like "%error%" group by user_id, session_id) as errors \
on t.user_id = errors.user_id and t.session_id = errors.session_id where t.event_type = "page" and (t.timestamp < errors.error_time or errors.error_time is null)')

In [81]:
routes_sql.registerTempTable('routes')

In [82]:
result_sql = se.sql('select route, count(*) as count from (select user_id, session_id, concat_ws("-", collect_list(event_page)) as route \
from (select * from routes where previous_page is null or previous_page != event_page) group by user_id, session_id) \
group by route order by count desc limit 30').collect()

In [83]:
with open('result_sql.txt', 'w') as f:
    for line in result_sql:
         f.write("{}\t{}\n".format(line[0], line[1]))

### Spark RDD solution

In [109]:
rdd = df.rdd

In [110]:
errors_rdd = rdd.filter(lambda x: 'error' in x[2]).map(lambda x: (x[0] + '_' + x[1], int(x[4]))).reduceByKey(min)

In [111]:
rdd = rdd.filter(lambda x: x[2] == 'page').map(lambda x: (x[0] + '_' + x[1], (x[3], int(x[4])))).leftOuterJoin(errors_rdd)

In [112]:
rdd = rdd.filter(lambda x: (x[1][1] is None) or (x[1][1] > x[1][0][1]))

In [113]:
def create_route(pages):
    pages = list(pages)
    return '-'.join([x for i, x in enumerate(pages) if i == 0 or x != pages[i-1]])

In [115]:
routes_rdd = rdd.sortBy(lambda x: [x[0], x[1][0][1]]).map(lambda x: (x[0], x[1][0][0])).groupByKey().map(lambda x: (x[0], create_route(x[1])))

In [116]:
result_rdd = routes_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False).take(30)

In [117]:
with open('result_rdd.txt', 'w') as f:
    for line in result_rdd:
         f.write("{}\t{}\n".format(line[0], line[1]))